# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

Nath\*, Mathis\* et al. Using DeepLabCut for 3D markerless pose estimation during behavior across species.

pre-print: https://www.biorxiv.org/content/10.1101/476531v1

#### 1. Setup and parameters
requires: 2 c-arm videos per run folder, "training" folder with 2D distorted marker exports in experiment root

In [4]:
import warnings
warnings.simplefilter('ignore')
import deeplabcut
import pandas as pd
import ruamel.yaml
import sys
import os
import tensorflow as tf
import numpy as np
import re

wd=r"E:\Users\Phil\DeepLabCut\dev"
vid_directory=r"Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat"
csv_directory = os.path.join(vid_directory,"training")
experimenter='Phil'
markerlist = ['Body_ds1_crn',
              'Body_ds2_int',
              'Body_ds3_cdl',
              'Body_vn1_crn',
              'Body_vn2_int',
              'Body_vn3_cdl',
              'Body_acc',
              'Scapula_acr',
              'Scapula_spi',
              'Scapula_vtb',
              'Scapula_acc',
              'Humerus_dpc',
              'Humerus_ent',
              'Humerus_ect',
              'Humerus_acc',
              'Ulna_olc',
              'Ulna_int',
              'Ulna_dst',
              'Ulna_acc',
              'Radius_prx',
              'Radius_int',
              'Radius_dst',
              'Radius_acc',
              'Teres_maj_prx',
              'Teres_maj_dst',
              'Biceps_prx',
              'Biceps_dst']
dotsize = 5
corner2move2 = 512

#### 2. Rename c-arm videos with trial name for easier id

In [43]:
def renameVideos(directory,skip=""):
    vid_list=[]
    for root, dirs, files in os.walk(directory):
        for name in files:
            if name.endswith(".avi") and not re.search(skip,root):
                run_name = root.split("\\")[-1]
                new_name = run_name+name.replace("era No.","")
                old_path = os.path.join(root,name)
                new_path = os.path.join(root,new_name)
                os.rename(old_path,new_path)
                vid_list.append(new_path)
            else:
                continue
    return vid_list
                
vid_list = renameVideos(vid_directory,"cals")

#### 3. Create new project (1 per animal)

In [45]:
task='1_3_4_5_unenhanced'
# vid_list=[ vid_directory+r'\run1retpro1_90-35-4ms\run1retpro1_90-35-4msCam1.avi',
#         vid_directory+r'\run1retpro1_90-35-4ms\run1retpro1_90-35-4msCam2.avi',
#         vid_directory+r'\run3circles_90-35-4ms\run3circles_90-35-4msCam1.avi',
#         vid_directory+r'\run3circles_90-35-4ms\run3circles_90-35-4msCam2.avi',
#         vid_directory+r'\run4addabd_90-35-4ms\run4addabd_90-35-4msCam1.avi',
#         vid_directory+r'\run4addabd_90-35-4ms\run4addabd_90-35-4msCam2.avi',
#         vid_directory+r'\run5flexextLAR_90-35-4ms\run5flexextLAR_90-35-4msCam1.avi',
#         vid_directory+r'\run5flexextLAR_90-35-4ms\run5flexextLAR_90-35-4msCam2.avi'
#       ]
path_config_file=deeplabcut.create_new_project(task,experimenter,vid_list, working_directory=wd,copy_videos=False) 

Created "E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\videos"
Created "E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\labeled-data"
Created "E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\training-datasets"
Created "E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\dlc-models"
Creating the symbolic link of the video
Created the symlink of Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run1retpro1_90-35-4ms\run1retpro1_90-35-4msCam1.avi to E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\videos\run1retpro1_90-35-4msCam1.avi
Created the symlink of Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat\run1retpro1_90-35-4ms\run1retpro1_90-35-4msCam2.avi to E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\videos\run1retpro1_90-35-4msCam2.avi
Created the symlink of Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\23Ap

#### 4. Overwrite default bodyparts with XROMM marker list

In [7]:
%%capture
config = ruamel.yaml.load(open(path_config_file))
config['bodyparts']=markerlist
config['dotsize']=dotsize
config['corner2move2']=[corner2move2,corner2move2]
ruamel.yaml.round_trip_dump(config, sys.stdout)
with open(path_config_file, 'w') as fp:
    ruamel.yaml.round_trip_dump(config, fp)
    fp.close()

#### 5. Extract frames from vid_list for training set

In [47]:
%matplotlib inline
deeplabcut.extract_frames(path_config_file, userfeedback=False) 

Config file read successfully.
Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 26.67  seconds.
Extracting and downsampling... 800  frames from the video.


800it [00:08, 90.60it/s] 


Kmeans clustering ... (this might take a while)
Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 26.67  seconds.
Extracting and downsampling... 800  frames from the video.


800it [00:09, 81.47it/s]


Kmeans clustering ... (this might take a while)
Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 26.67  seconds.
Extracting and downsampling... 800  frames from the video.


...

Kmeans clustering ... (this might take a while)
Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 26.67  seconds.
Extracting and downsampling... 800  frames from the video.


800it [00:11, 71.58it/s]


Kmeans clustering ... (this might take a while)
Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 26.67  seconds.
Extracting and downsampling... 800  frames from the video.


800it [00:09, 80.77it/s]


Kmeans clustering ... (this might take a while)

Frames were selected.
You can now label the frames using the function 'label_frames' (if you extracted enough frames for all videos).


#### 6. Convert XMALab exports to DeepLabCut format
No spaces in marker names, otherwise 2D export fails (more header columns than data)


In [11]:
run_names = np.unique(np.array([vid.split("\\")[-2] for vid in vid_list]))
labeled_data_path=os.path.join(path_config_file.split('config.yaml')[0],"labeled-data\\")

def hflip(x_old, width):
    x_flipped = width - 1 - x_old
    return x_flipped

def xmalab2dlc(run,csv_directory,labeled_data_path, width=1024, h_flip=False):  
    ## import XMAlab 2D exports
    df = pd.read_csv(csv_directory+"\\"+run+".csv", sep=',', header=0, dtype='float', na_values=' NaN ')
    ## coerce data into DeepLabCut hierarchical format
    df['frame_index']=df.index
    df['scorer']=experimenter
    df = df.melt(id_vars=['frame_index','scorer'])
    new = df['variable'].str.rsplit("_",n=2,expand=True)
    df['variable'],df['cam'],df['coords'] = new[0], new[1], new[2]
    df=df.rename(columns={'variable':'bodyparts'})
    df['coords']=df['coords'].str.rstrip(" ").str.lower()
    if h_flip == True:
        df['value'][df['coords']=='x']= df['value'][df['coords']=='x'].apply(lambda x:width-1-x)
    df['bodyparts']=df['bodyparts'].str.lstrip(" ").astype("category")
    df['bodyparts'].cat.set_categories(markerlist,inplace=True)
    df['frame_index'] = ['labeled-data\\' + run+"Cam"+x[-1] + '\\img' + (f"{y:03d}") + '.png' for x, y in zip(df['cam'], df['frame_index'])]
    newdf = df.pivot_table(columns=['scorer', 'bodyparts', 'coords'],index='frame_index',values='value',aggfunc='first',dropna=False)
    newdf.index.name=None
    ## go into frame folders and get frame index ##
    extracted_frames = []
    for root, dirs, files in os.walk(labeled_data_path):
        for name in files:
            if name.endswith(".png") and run in root:
                camera_id = root.split(' ')[-1][-1]
                frame_no = int(name.split('.')[0].replace('img',''))
                new_name = 'labeled-data\\'+run+"Cam"+camera_id+'\\img' + (f"{frame_no:03d}") + '.png'
                extracted_frames.append(new_name)

    ## filter by list of extracted frames
    df_extracted = newdf.filter(items=pd.Index(extracted_frames),axis=0)

    ## split new df into cams 1 and 2
    df1 = df_extracted.filter(like=run+"Cam"+"1",axis=0)
    df2 = df_extracted.filter(like=run+"Cam"+"2",axis=0)

    ## split new df into cams 1 and 2, export as h5 and csv
    for x in [1,2]:
        cam_name = run+"Cam"+str(x)
        dfx = df_extracted.filter(like=cam_name,axis=0)
        data_name = labeled_data_path+cam_name+"\\CollectedData_"+experimenter+".h5"
        print(data_name)
        dfx.to_hdf(data_name, 'df_with_missing', format='table', mode='w')
        dfx.to_csv(data_name.split('.h5')[0]+'.csv')
        print("saved "+str(data_name))

        
for run in run_names:
    xmalab2dlc(run,csv_directory,labeled_data_path, h_flip=True)

E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\labeled-data\run1retpro1_90-35-4msCam1\CollectedData_Phil.h5
saved E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\labeled-data\run1retpro1_90-35-4msCam1\CollectedData_Phil.h5
E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\labeled-data\run1retpro1_90-35-4msCam2\CollectedData_Phil.h5
saved E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\labeled-data\run1retpro1_90-35-4msCam2\CollectedData_Phil.h5
E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\labeled-data\run3circles_90-35-4msCam1\CollectedData_Phil.h5
saved E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\labeled-data\run3circles_90-35-4msCam1\CollectedData_Phil.h5
E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\labeled-data\run3circles_90-35-4msCam2\CollectedData_Phil.h5
saved E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\labeled-data\run3circles_90-35-4msCam

#### 7. Check substituted labels

In [12]:
##UNCOMMENT 2 LINES TO MANUALLY LABEL WITH GUI
# %gui wx
# deeplabcut.label_frames(path_config_file)
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by Phil.
E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\labeled-data\run1retpro1_90-35-4msCam1_labeled  already exists!
They are stored in the following folder: E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\labeled-data\run1retpro1_90-35-4msCam1_labeled.
E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\labeled-data\run1retpro1_90-35-4msCam2_labeled  already exists!
They are stored in the following folder: E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\labeled-data\run1retpro1_90-35-4msCam2_labeled.
E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\labeled-data\run3circles_90-35-4msCam1_labeled  already exists!
They are stored in the following folder: E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\labeled-data\run3circles_90-35-4msCam1_labeled.
E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\labeled-data\run3circles_90-35-4msCam2_labeled  already 

#### 8. Create training set

In [13]:
deeplabcut.create_training_dataset(path_config_file)

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


#### 9. Start training

In [ ]:
deeplabcut.train_network(path_config_file, displayiters=50,saveiters=10000)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24],
                [25],
                [26]],
 'all_joints_names': ['Body_ds1_crn',
                      'Body_ds2_int',
                      'Body_ds3_cdl',
                      'Body_vn1_crn',
                      'Body_vn2_int',
                      'Body_vn3_cdl',
                      'Body_acc',
                      'Scapula_acr',
                      'Scapula_spi',
                      'Scapula_vtb',
                      'Scapula_acc

Starting with standard pose-dataset loader.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\LabAdmin\.conda\envs\dlc-windowsGPU\lib\site-packages\deeplabcut\pose_estimation_tensorflow\models\pretrained\resnet_v1_50.ckpt
Display_iters overwritten as 50
Save_iters overwritten as 10000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'E:\\Users\\Phil\\DeepLabCut\\dev\\1_3_4_5_unenhanced-Phil-2019-09-30\\dlc-models\\iteration-0\\1_3_4_5_unenhancedSep30-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05

iteration: 50 loss: 0.1687 lr: 0.005
iteration: 100 loss: 0.0289 lr: 0.005
iteration: 150 loss: 0.0266 lr: 0.005
iteration: 200 loss: 0.0231 lr: 0.005
iteration: 250 loss: 0.0245 lr: 0.005
iteration: 300 loss: 0.0241 lr: 0.005
iteration: 350 loss: 0.0221 lr: 0.005
iteration: 400 loss: 0.0239 lr: 0.005
iteration: 450 loss: 0.0213 lr: 0.005
iteration: 500 loss: 0.0219 lr: 0.005
iteration: 550 loss: 0.0228 lr: 0.005
iteration: 600 loss: 0.0210 lr: 0.005
iteration: 650 loss: 0.0198 lr: 0.005
iteration: 700 loss: 0.0189 lr: 0.005
iteration: 750 loss: 0.0180 lr: 0.005
iteration: 800 loss: 0.0184 lr: 0.005
iteration: 850 loss: 0.0178 lr: 0.005
iteration: 900 loss: 0.0169 lr: 0.005
iteration: 950 loss: 0.0182 lr: 0.005
iteration: 1000 loss: 0.0181 lr: 0.005
iteration: 1050 loss: 0.0159 lr: 0.005
iteration: 1100 loss: 0.0159 lr: 0.005
iteration: 1150 loss: 0.0170 lr: 0.005


In [ ]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

In [ ]:
videofile_path = [r'\\tsclient\Downloads\run1retpro'] #Enter a folder OR a list of videos to analyze.

deeplabcut.analyze_videos(path_config_file,videofile_path, videotype='.avi')

In [ ]:
deeplabcut.create_labeled_video(path_config_file,[r'\\tsclient\Downloads\run1retpro'])

## Extract outlier frames [optional step]

This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. This step has many options, so please look at:

In [ ]:
deeplabcut.extract_outlier_frames?

In [40]:
deeplabcut.create_labeled_video?

Signature: deeplabcut.create_labeled_video(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, filtered=False, save_frames=False, Frames2plot=None, delete=False, displayedbodyparts='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False)
Docstring:
    Labels the bodyparts in a video. Make sure the video is already analyzed by the function 'analyze_video'

    Parameters
    ----------
    config : string
        Full path of the config.yaml file as a string.

    videos : list
        A list of strings containing the full paths to videos for analysis or a path to the directory, where all the videos with same extension are stored.
    
    videotype: string, optional
        Checks for the extension of the video in case the input to the video is a directory.
 Only videos with this extension are analyzed. The default is ``.avi``

    shuffle : int, optional
        Number of shuffles of training dataset. Default is

In [ ]:
deeplabcut.extract_outlier_frames(path_config_file,['/videos/video3.avi']) #pass a specific video

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset. 

In [ ]:
%gui wx
deeplabcut.refine_labels(path_config_file)

**NOTE:** Afterwards, if you want to look at the adjusted frames, you can load them in the main GUI by running: ``deeplabcut.label_frames(path_config_file)``

(you can add a new "cell" below to add this code!)

#### Once all folders are relabeled, check the labels again! If you are not happy, adjust them in the main GUI:

``deeplabcut.label_frames(path_config_file)``

Check Labels:

``deeplabcut.check_labels(path_config_file)``

In [ ]:
#NOW, merge this with your original data:

deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]
Following the refinement of labels and appending them to the original dataset, this creates a new iteration of training dataset. This is automatically set in the config.yaml file, so let's get training!

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

THIS HAS MANY FUN OPTIONS! 

``deeplabcut.create_labeled_video(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, filtered=False, save_frames=False, Frames2plot=None, delete=False, displayedbodyparts='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False)``

So please check:

In [4]:
deeplabcut.create_labeled_video?

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)